In [30]:
import os
from pathlib import Path
from typing import List, Optional
import ftplib
import logging
from tenacity import retry, stop_after_attempt, wait_exponential
# from qrlib.QRComponent import QRComponent
# from qrlib.QREnv import QREnv

logger = logging.getLogger(__name__)

class FTPError(Exception):
    """Base exception for FTP-related errors"""
    pass

class FTPConnectionError(FTPError):
    """Exception for FTP connection issues"""
    pass

class FTPAuthenticationError(FTPError):
    """Exception for FTP authentication issues"""
    pass

class FTPOperationError(FTPError):
    """Exception for FTP operation issues"""
    pass

class FTPComponent():
    """Component for handling FTP operations"""

    def __init__(self, local_download_path: str, ftp_working_dir: str):
        """Initialize FTP component
        
        Args:
            local_download_path: Local path for downloading files
            ftp_working_dir: Working directory on FTP server
        """
        super().__init__()
        self.__ftp: Optional[ftplib.FTP] = None
        self.__server: str = ""
        self.__port: int = 21
        self.__username: str = ""
        self.__password: str = ""
        self.__initial_dir: str = ""
        self.local_download_path = Path(local_download_path)
        self.ftp_working_dir = ftp_working_dir
        
        # Create download directory if it doesn't exist
        self.local_download_path.mkdir(parents=True, exist_ok=True)

    def load_ftp_vault(self) -> None:
        """Load FTP credentials from vault"""
        try:
            # vault = QREnv.VAULTS.get("ftp_credentials")
            vault = {
                "server": "192.168.101.12",
                "port": 21,
                "username": "",
                "password": "",
                "initial_dir": "/letteraction"
            }
            if not vault:
                raise FTPError("FTP vault not found")

            self.__server = vault.get("server", "192.168.101.12")
            self.__port = int(vault.get("port", 21))
            self.__username = vault.get("username", "")
            self.__password = vault.get("password", "")
            self.__initial_dir = vault.get("initial_dir", "/letteraction")

            # Validate credentials
            # if not all([self.__server, self.__username, self.__password]):
            #     raise FTPError("Missing required FTP credentials")

            logger.info("Successfully loaded FTP credentials")

        except Exception as e:
            logger.error(f"Failed to load FTP credentials: {str(e)}")
            raise FTPError(f"Failed to load FTP credentials: {str(e)}")

    def __enter__(self):
        """Context manager entry"""
        self.connect()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit"""
        self.disconnect()
        if exc_type:
            logger.error(f"Error during FTP operation: {str(exc_val)}")
            return False
        return True

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=4, max=10),
        reraise=True
    )
    def connect(self) -> None:
        """Connect to FTP server with retry logic"""
        try:
            logger.info(f"Connecting to FTP server {self.__server}")
            self.__ftp = ftplib.FTP()
            self.__ftp.connect(
                host=self.__server,
                port=self.__port,
                timeout=30
            )
            self.__ftp.login(
                user=self.__username,
                passwd=self.__password
            )
            
            # Set initial directory if specified
            if self.__initial_dir:
                self.__ftp.cwd(self.__initial_dir)
            
            logger.info(f"Successfully connected to FTP server: {self.__ftp.welcome}")
            
        except ftplib.error_perm as e:
            error_msg = str(e)
            if "530" in error_msg:
                raise FTPAuthenticationError("Invalid FTP credentials")
            elif "550" in error_msg:
                raise FTPOperationError(f"FTP operation failed: {error_msg}")
            else:
                raise FTPError(f"FTP permission error: {error_msg}")
                
        except ftplib.error_temp as e:
            raise FTPConnectionError(f"Temporary FTP error: {str(e)}")
            
        except ftplib.error_proto as e:
            raise FTPError(f"FTP protocol error: {str(e)}")
            
        except Exception as e:
            raise FTPConnectionError(f"Failed to connect to FTP server: {str(e)}")

    def disconnect(self) -> None:
        """Disconnect from FTP server"""
        if self.__ftp:
            try:
                self.__ftp.quit()
            except:
                self.__ftp.close()
            finally:
                self.__ftp = None
                logger.info("Disconnected from FTP server")

    def set_cwd(self, directory: str) -> None:
        """Change working directory on FTP server
        
        Args:
            directory: Directory to change to
        """
        if not self.__ftp:
            raise FTPError("Not connected to FTP server")
            
        try:
            self.__ftp.cwd(directory)
            logger.info(f"Changed directory to: {directory}")
        except ftplib.error_perm as e:
            raise FTPOperationError(f"Failed to change directory: {str(e)}")

    def reset_wd(self) -> None:
        """Reset to initial working directory"""
        if self.__initial_dir:
            self.set_cwd(self.__initial_dir)

    def list_all_files(self, pattern: str = "*.pdf") -> List[str]:
        """List all files in current directory matching pattern
        
        Args:
            pattern: File pattern to match (default: *.pdf)
            
        Returns:
            List of matching filenames
        """
        if not self.__ftp:
            raise FTPError("Not connected to FTP server")

        try:
            # Get all files
            files = []
            self.__ftp.retrlines('NLST', files.append)
            
            # Filter by pattern if specified
            if pattern != "*":
                from fnmatch import fnmatch
                files = [f for f in files if fnmatch(f, pattern)]
                
            logger.info(f"Found {len(files)} files matching pattern: {pattern}")
            return files
            
        except ftplib.error_perm as e:
            raise FTPOperationError(f"Failed to list files: {str(e)}")

    def download_file(self, remote_file: str, local_file: Optional[str] = None) -> Path:
        """Download file from FTP server
        
        Args:
            remote_file: Name of file on FTP server
            local_file: Local filename (optional)
            
        Returns:
            Path to downloaded file
        """
        if not self.__ftp:
            raise FTPError("Not connected to FTP server")

        local_path = self.local_download_path / (local_file or remote_file)
        
        try:
            # Ensure parent directory exists
            local_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Download file
            with open(local_path, 'wb') as f:
                self.__ftp.retrbinary(f'RETR {remote_file}', f.write)
                
            logger.info(f"Downloaded {remote_file} to {local_path}")
            return local_path
            
        except ftplib.error_perm as e:
            raise FTPOperationError(f"Failed to download file: {str(e)}")
        except Exception as e:
            raise FTPError(f"Error downloading file: {str(e)}")

    def upload_file(self, local_path: str, remote_file: str) -> None:
        """Upload file to FTP server
        
        Args:
            local_path: Path to local file
            remote_file: Name for file on FTP server
        """
        if not self.__ftp:
            raise FTPError("Not connected to FTP server")

        try:
            with open(local_path, 'rb') as f:
                self.__ftp.storbinary(f'STOR {remote_file}', f)
            logger.info(f"Uploaded {local_path} to {remote_file}")
            
        except ftplib.error_perm as e:
            raise FTPOperationError(f"Failed to upload file: {str(e)}")
        except Exception as e:
            raise FTPError(f"Error uploading file: {str(e)}")

    def create_directory(self, directory: str) -> None:
        """Create directory on FTP server if it doesn't exist
        
        Args:
            directory: Directory to create
        """
        if not self.__ftp:
            raise FTPError("Not connected to FTP server")

        try:
            try:
                self.__ftp.cwd(directory)
            except ftplib.error_perm:
                self.__ftp.mkd(directory)
                logger.info(f"Created directory: {directory}")
        except ftplib.error_perm as e:
            raise FTPOperationError(f"Failed to create directory: {str(e)}")

    def delete_file(self, remote_file: str) -> None:
        """Delete file from FTP server
        
        Args:
            remote_file: Name of file to delete
        """
        if not self.__ftp:
            raise FTPError("Not connected to FTP server")

        try:
            self.__ftp.delete(remote_file)
            logger.info(f"Deleted file: {remote_file}")
        except ftplib.error_perm as e:
            raise FTPOperationError(f"Failed to delete file: {str(e)}")

    def move_file(self, source: str, destination: str) -> None:
        """Move/rename file on FTP server
        
        Args:
            source: Source filename
            destination: Destination filename
        """
        if not self.__ftp:
            raise FTPError("Not connected to FTP server")

        try:
            self.__ftp.rename(source, destination)
            logger.info(f"Moved {source} to {destination}")
        except ftplib.error_perm as e:
            raise FTPOperationError(f"Failed to move file: {str(e)}")


In [31]:
ftp = FTPComponent(
    local_download_path=r'/Users/saurabshrestha/Documents/Kumari/LetterAction/LetterAction-credentials/LetterAction/local_folder',
    ftp_working_dir=r''
)

In [ ]:
ftp.load_ftp_vault()

with ftp as ftp:
    ftp.set_cwd(r'scanned_documents')
    logger.info("Set ftp folder to source folder")
    file_list = ftp.list_all_files()
    print(ftp.check_pwd())
    for file in file_list:
        ftp.download_file(file)
    logger.info(f"No of file found is {len(file_list)}")
    ftp.reset_wd()

In [ ]:
print(file_list)

['797--20241219044306.pdf']
